# Introduction

## Major imports

In [51]:
import pandas as pd
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [52]:
df = pd.read_csv(filepath_or_buffer='ncses_table_srv_data_NSCG_2024-11-11T00_38_01Z\\ncses_table_raw_data_NSCG_2024-11-11T00_38_01Z.csv')
df.dropna(how='all',axis='index')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 6 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Year                                                470 non-null    int64 
 1   Labor force status                                  470 non-null    object
 2   Race (2003-2021)                                    470 non-null    object
 3   Gender                                              470 non-null    object
 4   Field of maj - most recent deg (major) (2003-2021)  470 non-null    object
 5   Weighted Count                                      470 non-null    object
dtypes: int64(1), object(5)
memory usage: 22.2+ KB


In [53]:
#Saving old frame just in case
df_old = df
races = df['Race (2003-2021)'].unique()
df = df.replace(to_replace=races, value= list(range(0, len(races))))
df

C:\Users\pauli\AppData\Local\Temp\ipykernel_130176\2632857430.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(to_replace=races, value= list(range(0, len(races))))


,Year,Labor force status,Race (2003-2021),Gender,Field of maj - most recent deg (major) (2003-2021),Weighted Count
0,2021,Employed,0,Female,Computer and mathematical sciences,"280,222"
1,2021,Employed,0,Female,"Biological, agricultural and environmental lif...","197,031"
2,2021,Employed,0,Female,Physical and related sciences,"45,603"
3,2021,Employed,0,Female,Social and related sciences,"259,090"
4,2021,Employed,0,Female,Engineering,"213,556"
...,...,...,...,...,...,...
465,2019,Not In Labor Force,5,Male,Physical and related sciences,604
466,2019,Not In Labor Force,5,Male,Social and related sciences,"8,177"
467,2019,Not In Labor Force,5,Male,Engineering,"5,593"
468,2019,Not In Labor Force,5,Male,S&E-Related Fields,"12,595"


In [54]:
X = df.drop(columns = 'Labor force status')
y = df['Labor force status']


In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=746, test_size=0.3)
X_train

,Year,Race (2003-2021),Gender,Field of maj - most recent deg (major) (2003-2021),Weighted Count
86,2021,0,Female,Physical and related sciences,"2,442"
105,2021,1,Male,Social and related sciences,436
233,2021,5,Male,Social and related sciences,"14,205"
394,2019,0,Female,Physical and related sciences,"13,123"
38,2021,2,Male,Social and related sciences,"206,105"
...,...,...,...,...,...
120,2021,2,Male,Engineering,"9,838"
21,2021,1,Male,Computer and mathematical sciences,"3,935"
96,2021,0,Male,S&E-Related Fields,"12,408"
261,2019,1,Male,Social and related sciences,"14,891"


In [56]:
model = linear_model.LogisticRegression()
model.fit(X_train,y_train)

ValueError: could not convert string to float: 'Female'